In [7]:
import pandas as pd
import json
import rdkit as rd 
import pymatgen as pm
from pymatgen.core import Structure

In [21]:
qmof_attrs = pd.read_csv("/scratch/saigum/qmof_database/qmof_database/qmof.csv")

/tmp/ipykernel_8083/3081914008.py:1: DtypeWarning: Columns (40,41,43,44,45,46,47,49,50,51,52,53,55,56,57,65,66,67,68,69,77,78,79,80,81,89,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  qmof_attrs = pd.read_csv("/scratch/saigum/qmof_database/qmof_database/qmof.csv")


In [ ]:
with open("/scratch/saigum/qmof_database/qmof_database/qmof_structure_data.json") as f:
    struct_data = json.load(f)
qmof_structs = {}
for entry in struct_data:
    qmof_structs[entry['qmof_id']] = Structure.from_dict(entry['structure'])

In [ ]:

class QMOF():
    def __init__(self,root_path:str):
        with open(f"{root_path}/qmof_structure_data.json") as f:
            STRUCTURE_DATA = json.load(f)
            self.ID2NAME = {STRUCTURE_DATA[i]["qmof_id"]:STRUCTURE_DATA[i]["name"]
                               for i in range(len(STRUCTURE_DATA)) }
            self.STRUCTURE_DATA = {STRUCTURE_DATA[i]["qmof_id"]:STRUCTURE_DATA[i]["structure"]
                               for i in range(len(STRUCTURE_DATA)) }
        
            
    def get_graph(self,mol_id):
        STRUCTURE = Structure.from_dict(self.STRUCTURE_DATA[mol_id]["structure"])
        ## now to construct node featurues
        coords = STRUCTURE.cart_coords
        



In [2]:
import json
fp =open("/scratch/saigum/qmof_database/qmof_database/qmof_structure_data.json","r") 
STRUCTURE_DATA = json.load(fp)

In [8]:
structure = Structure.from_dict(STRUCTURE_DATA[0]["structure"])

In [9]:
structure.cart_coords

array([[ 1.22089,  5.19252,  4.035  ],
       [ 6.1393 ,  1.87981,  7.94192],
       [ 0.78236,  7.07112,  9.75828],
       [ 4.62932,  6.39495,  2.83247],
       [ 2.73088,  0.67737,  9.14446],
       [ 0.31647,  2.58806,  1.15508],
       [ 7.04372,  4.48423, 10.8218 ],
       [ 3.31973,  2.81282,  2.49573],
       [ 4.04046,  4.2595 ,  9.48113],
       [ 2.3254 ,  0.40517,  4.56399],
       [ 5.03481,  6.66717,  7.41299],
       [ 2.70658,  6.55707,  6.76615],
       [ 4.6536 ,  0.51526,  5.21081],
       [ 2.79833,  2.45871,  4.87236],
       [ 4.56186,  4.61359,  7.10456],
       [ 2.62076,  2.86113,  6.37909],
       [ 4.73942,  4.21126,  5.59782],
       [ 4.91868,  5.71021,  3.65155],
       [ 2.44153,  1.36212,  8.32538],
       [ 0.51119,  1.94678,  2.03696],
       [ 6.84901,  5.12555,  9.93994],
       [ 4.01548,  1.98852,  2.76232],
       [ 3.3447 ,  5.08381,  9.21457],
       [ 6.14282,  5.76308,  4.04245],
       [ 1.21739,  1.30925,  7.93447],
       [ 4.06548,  4.9278

In [ ]:
with open('qmof_structure_data.json') as f:
    struct_data = json.load(f)
qmof_structs = {}
for entry in struct_data:
    qmof_structs[entry['qmof_id']] = Structure.from_dict(entry['structure'])

Structure Summary
Lattice
    abc : 6.9419523112540436 7.17878940055673 8.791651649033692
 angles : 82.25599709677074 71.35701701555689 81.58285544631552
 volume : 408.857471262788
      A : np.float64(6.577866356) np.float64(0.001198396) np.float64(2.218642526)
      B : np.float64(0.78166899) np.float64(7.070336339) np.float64(0.96662035)
      C : np.float64(0.000658751) np.float64(0.000799513) np.float64(8.791651588)
    pbc : True True True
PeriodicSite: Ba (1.221, 5.193, 4.035) [0.0983, 0.7344, 0.3534]
PeriodicSite: Ba (6.139, 1.88, 7.942) [0.9017, 0.2656, 0.6466]
PeriodicSite: Cu (0.7824, 7.071, 9.758) [5.171e-06, 1.0, 1.0]
PeriodicSite: H (4.629, 6.395, 2.832) [0.5963, 0.9044, 0.07226]
PeriodicSite: H (2.731, 0.6774, 9.144) [0.4037, 0.09563, 0.9277]
PeriodicSite: H (0.3165, 2.588, 1.155) [0.004605, 0.366, 0.08998]
PeriodicSite: H (7.044, 4.484, 10.82) [0.9954, 0.634, 0.91]
PeriodicSite: H (3.32, 2.813, 2.496) [0.4574, 0.3977, 0.1247]
PeriodicSite: H (4.04, 4.26, 9.481) [0.5426,

In [ ]:
pm.Structure()

In [ ]:
def mol2graph(moljson):
    

In [ ]:
cp = open("/scratch/saigum/qmof_database/qmof_database/qmof.json","r")
MOF_DATA = json.load(cp)
print(STRUCTURE_DATA)

In [ ]:
print(MOF_DATA)

In [ ]:
def mol2graph()